In [3]:
!pip install -q -U langchain_core langgraph langchain_google_genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.9/437.9 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 20.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [4]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get("GOOGLE_API_KEY")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    api_key=GEMINI_API_KEY,
    model="gemini-2.0-flash"
)
llm.invoke("hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--3e5c2a65-59cf-4cde-a26c-5deeca03cfab-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [5]:
def multiply(a,b):
  """Multiply a and b
  return a *b
  """
  return a * b
def add(a,b):
  """Add a and b
  return a + b
  """
  return a + b
def subtract(a,b):
  """Subtract a and b
  return a - b
  """
  return a - b
def divide(a,b):
  """Divide a and b
  return a / b
  """
  return a / b
tools = [multiply,add,subtract,divide]
llm_with_tools = llm.bind_tools(tools)

In [6]:
from langchain_core.messages import SystemMessage,HumanMessage
from langgraph.graph.message import MessagesState
system_message = SystemMessage(content="You are a helpful assistant. which do only arthematic operation on provided input")
def assistant(state:MessagesState):
  return {"messages":[llm_with_tools.invoke([system_message] + state["messages"])]}

In [7]:
from langgraph.graph import START,StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
builder = StateGraph(MessagesState)
builder.add_node("assistant",assistant)
builder.add_node("tools",ToolNode(tools))
builder.add_edge(START,"assistant")
builder.add_conditional_edges("assistant",tools_condition)
builder.add_edge("tools","assistant")
graph = builder.compile()

In [8]:
messages = graph.invoke({"messages":[HumanMessage(content="what is 2 + 2 -2 /1*12")]})
from pprint import pprint
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

what is 2 + 2 -2 /1*12
================================== Ai Message ==================================
Tool Calls:
  add (910a5ce6-4173-4670-853f-d11d40a328e0)
 Call ID: 910a5ce6-4173-4670-853f-d11d40a328e0
  Args:
    a: 2.0
    b: 2.0
================================= Tool Message =================================
Name: add

4.0
================================== Ai Message ==================================
Tool Calls:
  subtract (778874fc-c97a-43a3-b78b-03fc70775888)
 Call ID: 778874fc-c97a-43a3-b78b-03fc70775888
  Args:
    a: 4.0
    b: 2.0
================================= Tool Message =================================
Name: subtract

2.0
================================== Ai Message ==================================
Tool Calls:
  divide (12561b74-a495-41a3-a675-1a7809ceec20)
 Call ID: 12561b74-a495-41a3-a675-1a7809ceec20
  Args:
    a: 2.0
    b: 1.0
================================= Tool Messag

In [10]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()
graph_memory = builder.compile(checkpointer=memory)
config = {"configurable":{"thread_id":"1"}}
messages = [HumanMessage(content="what is 2 + 2")]
messages = graph_memory.invoke({"messages":messages},config=config)
from pprint import pprint
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

what is 2 + 2
================================== Ai Message ==================================
Tool Calls:
  add (d066c0b1-de70-4c9a-9bae-2aeec11de45e)
 Call ID: d066c0b1-de70-4c9a-9bae-2aeec11de45e
  Args:
    a: 2.0
    b: 2.0
================================= Tool Message =================================
Name: add

4.0
================================== Ai Message ==================================

2 + 2 = 4


In [11]:
graph_memory.invoke({"messages":[HumanMessage(content="No add 10")]},config=config)

{'messages': [HumanMessage(content='what is 2 + 2', additional_kwargs={}, response_metadata={}, id='2e1f240f-365d-4744-be33-a3c112e93394'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 2.0, "b": 2.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--b9e5084e-5211-4562-b497-61c36401de28-0', tool_calls=[{'name': 'add', 'args': {'a': 2.0, 'b': 2.0}, 'id': 'd066c0b1-de70-4c9a-9bae-2aeec11de45e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 83, 'output_tokens': 5, 'total_tokens': 88, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='4.0', name='add', id='481782fc-74f3-4c32-80c3-2ee023769a17', tool_call_id='d066c0b1-de70-4c9a-9bae-2aeec11de45e'),
  AIMessage(content='2 + 2 = 4', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_rea